### Main imports and constants ###

In [1]:
import pandas as pd
import folium
from folium import plugins

In [2]:
# csv file
FILE_NAME = '2022_04_15_sample.csv'

# names of columns for 'filter' function
COLS_TO_CONCATENATE = ['vhl_name', 'ol', 'ob']

# tram icon
TRAM_ICON = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTo8ghk-iBncj_HnWVMjR623u97xyVwnGLmPg&usqp=CAU"

### Main functions ###

In [3]:
def get_unique_ids(data: pd.DataFrame, cols_to_concatenate: list) -> list:
    '''
    Get unique values from the dataframe, that can indicate a sigle bus/tram route
    For example: three numbers: brigade number, line number and vehicle number put together
    indicate that this is a single route because it is driven by the same bridge AND in the 
    exact vehicle AND the number displayed on this vehicle is the same.

    Arguments:
        data: pandas DataFrame,
        cols_to_concatenate: names of columns
    
    Returns:
        list of unique number sets, that show a sigle route, for example:
        ['3205_26_11', '3205_13_6']
    '''

    df = data[~data.isna().any(axis=1)] # data with no nan values
    df = df[cols_to_concatenate].astype(int).astype(str) # convert to int (no .0) and then to str

    return df[cols_to_concatenate].apply(lambda x: '_'.join(x), axis=1).unique().tolist()

In [4]:
def filter(data: pd.DataFrame, pattern: str) -> pd.DataFrame:
    '''
    Perform a query on the dataframe based on provided pattern.

    Arguments:
        data: pandas DataFrame,
        pattern: single result of get_unique_ids function
    
    Returns:
        filtered dataframe based on provided pattern
    '''
    # change pattern to dictionary
    look_up_dict = dict(zip(COLS_TO_CONCATENATE, pattern.split('_')))
    query_list = [] # make emtpy list
    for key in look_up_dict.keys():
        query_list.append(f'{key}=={look_up_dict[key]}')
    query = ' & '.join(query_list) # sql query for dataframe filtering
    return data.query(query)

In [5]:
def change_date(data: pd.DataFrame, date_col: str)-> pd.DataFrame:
    '''
    Add 'dates' column to data. This new column changes 'date_col' column
    to match folium data style

    Arguments:
        data: pandas DataFrame,
        date_col: name of column with dates
    
    Returns:
        datagrame with additional column
    '''
    data['dates'] = data[date_col].apply(lambda x: x.split('.')[0].replace(' ', 'T'))
    return data

### Map generation ###

In [6]:
# Load data file
data = pd.read_csv(FILE_NAME)

In [7]:
# Get patterns for individual routes
results = get_unique_ids(data, COLS_TO_CONCATENATE)
results

['3205_26_11', '3205_13_6']

In [8]:
# Pick a route 
# (in this case route 3205_26_11)

UNIQUE_ROUTE_NUMBER_ID = 1

new_df = filter(data, results[UNIQUE_ROUTE_NUMBER_ID])
new_df

,Unnamed: 0,vhl_namespace,vhl_name,lsh_time,lsh_added,lat,lon,ol,ob
485,485,tw,3205,2022-04-15 06:24:59+02,2022-04-15 06:25:03.012274+02,52.251663,21.038311,13.0,6.0
487,487,tw,3205,2022-04-15 06:25:04+02,2022-04-15 06:25:08.15755+02,52.251877,21.037950,13.0,6.0
488,488,tw,3205,2022-04-15 06:25:09+02,2022-04-15 06:25:13.762641+02,52.251976,21.037780,13.0,6.0
489,489,tw,3205,2022-04-15 06:25:15+02,2022-04-15 06:25:17.687828+02,52.251976,21.037779,13.0,6.0
490,490,tw,3205,2022-04-15 06:25:20+02,2022-04-15 06:25:24.14978+02,52.251976,21.037779,13.0,6.0
...,...,...,...,...,...,...,...,...,...
9910,9910,tw,3205,2022-04-15 20:47:25+02,2022-04-15 20:47:27.406145+02,52.257076,21.055296,13.0,6.0
9911,9911,tw,3205,2022-04-15 20:47:30+02,2022-04-15 20:47:32.996415+02,52.257076,21.055296,13.0,6.0
9912,9912,tw,3205,2022-04-15 20:47:36+02,2022-04-15 20:47:38.282766+02,52.257076,21.055296,13.0,6.0
9913,9913,tw,3205,2022-04-15 20:47:41+02,2022-04-15 20:47:43.834783+02,52.257080,21.055302,13.0,6.0


In [9]:
# Add data column to match folium data format
change_date(new_df, 'lsh_added')

c:\Users\mariu\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Unnamed: 0,vhl_namespace,vhl_name,lsh_time,lsh_added,lat,lon,ol,ob,dates
485,485,tw,3205,2022-04-15 06:24:59+02,2022-04-15 06:25:03.012274+02,52.251663,21.038311,13.0,6.0,2022-04-15T06:25:03
487,487,tw,3205,2022-04-15 06:25:04+02,2022-04-15 06:25:08.15755+02,52.251877,21.037950,13.0,6.0,2022-04-15T06:25:08
488,488,tw,3205,2022-04-15 06:25:09+02,2022-04-15 06:25:13.762641+02,52.251976,21.037780,13.0,6.0,2022-04-15T06:25:13
489,489,tw,3205,2022-04-15 06:25:15+02,2022-04-15 06:25:17.687828+02,52.251976,21.037779,13.0,6.0,2022-04-15T06:25:17
490,490,tw,3205,2022-04-15 06:25:20+02,2022-04-15 06:25:24.14978+02,52.251976,21.037779,13.0,6.0,2022-04-15T06:25:24
...,...,...,...,...,...,...,...,...,...,...
9910,9910,tw,3205,2022-04-15 20:47:25+02,2022-04-15 20:47:27.406145+02,52.257076,21.055296,13.0,6.0,2022-04-15T20:47:27
9911,9911,tw,3205,2022-04-15 20:47:30+02,2022-04-15 20:47:32.996415+02,52.257076,21.055296,13.0,6.0,2022-04-15T20:47:32
9912,9912,tw,3205,2022-04-15 20:47:36+02,2022-04-15 20:47:38.282766+02,52.257076,21.055296,13.0,6.0,2022-04-15T20:47:38
9913,9913,tw,3205,2022-04-15 20:47:41+02,2022-04-15 20:47:43.834783+02,52.257080,21.055302,13.0,6.0,2022-04-15T20:47:43


In [10]:
# create folium map and set starting point as first point in dataframe
m = folium.Map(location=new_df[['lat', 'lon']].iloc[0].tolist(), zoom_start=16)

# add geocoder (textbox to input geolocation names)
plugins.Geocoder().add_to(m)

# draw lines based on coordinates in dataframe
# 'lon' is first, 'lat' is second
lines = [
    {
        "coordinates": [[x,y] for x,y in zip(new_df['lon'], new_df['lat'])],
        "dates": new_df['dates'].tolist(),
        "color": "red"}]

# make 'features' to output them into TimestampedGeoJson
features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {
                "color": line["color"],
                },
            "icon": "marker",

            # comment whole 'iconstyle' for a default marker
            "iconstyle": { 
                "iconUrl": TRAM_ICON,
                "iconSize": [40, 40],
            },
        },
    }
    for line in lines
]

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1M",
    add_last_point=True,
).add_to(m)

In [11]:
m

In [12]:
m.save("test_tram_route.html")